### simple GENAI app using Lnagchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")

In [3]:
### Data Ingestion from the website, we need to scrape the data

from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader= WebBaseLoader("https://writer.com/blog/palmyra-llms-multilingual/?utm_source=google&utm_medium=paid-search&utm_campaign=multilingual&utm_content=multilingual-blog-post&utm_source=google&utm_medium=paid_search&utm_campaign=&utm_term=multilingual%20llm&utm_content=&hsa_acc=8779610535&hsa_cam=20957109982&hsa_grp=158242513895&hsa_ad=688126356904&hsa_src=g&hsa_tgt=kwd-2228024943017&hsa_kw=multilingual%20llm&hsa_mt=b&hsa_net=adwords&hsa_ver=3&gad_source=1&gclid=Cj0KCQjw99e4BhDiARIsAISE7P-8lapEODR3L9fXGY9r-xpddqrBDTD7xDQKVjKVolYNEU72v5i0WRcaAlA8EALw_wcB")
loader

In [5]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://writer.com/blog/palmyra-llms-multilingual/?utm_source=google&utm_medium=paid-search&utm_campaign=multilingual&utm_content=multilingual-blog-post&utm_source=google&utm_medium=paid_search&utm_campaign=&utm_term=multilingual%20llm&utm_content=&hsa_acc=8779610535&hsa_cam=20957109982&hsa_grp=158242513895&hsa_ad=688126356904&hsa_src=g&hsa_tgt=kwd-2228024943017&hsa_kw=multilingual%20llm&hsa_mt=b&hsa_net=adwords&hsa_ver=3&gad_source=1&gclid=Cj0KCQjw99e4BhDiARIsAISE7P-8lapEODR3L9fXGY9r-xpddqrBDTD7xDQKVjKVolYNEU72v5i0WRcaAlA8EALw_wcB', 'title': 'Introducing multilingual capabilities with Palmyra LLMs - Writer', 'description': 'Palmyra, the Writer-built family of LLMs, now supports highly-accurate text generation and translation in over 30 languages. Find out more.', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\nIntroducing multilingual capabilities with Palmyra LLMs - Writer\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [6]:
#### Load the data --> docs -> divide our doc into chunks --> vectors --> vector EMbedding --> vector storeDB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter =  RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(docs)

In [7]:
documents

[Document(metadata={'source': 'https://writer.com/blog/palmyra-llms-multilingual/?utm_source=google&utm_medium=paid-search&utm_campaign=multilingual&utm_content=multilingual-blog-post&utm_source=google&utm_medium=paid_search&utm_campaign=&utm_term=multilingual%20llm&utm_content=&hsa_acc=8779610535&hsa_cam=20957109982&hsa_grp=158242513895&hsa_ad=688126356904&hsa_src=g&hsa_tgt=kwd-2228024943017&hsa_kw=multilingual%20llm&hsa_mt=b&hsa_net=adwords&hsa_ver=3&gad_source=1&gclid=Cj0KCQjw99e4BhDiARIsAISE7P-8lapEODR3L9fXGY9r-xpddqrBDTD7xDQKVjKVolYNEU72v5i0WRcaAlA8EALw_wcB', 'title': 'Introducing multilingual capabilities with Palmyra LLMs - Writer', 'description': 'Palmyra, the Writer-built family of LLMs, now supports highly-accurate text generation and translation in over 30 languages. Find out more.', 'language': 'en-US'}, page_content='Introducing multilingual capabilities with Palmyra LLMs - Writer\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [9]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(documents,embeddings)

In [10]:
vectorstoredb

In [12]:
## QUREY From a vector DB
qurey="Shorten sales cycles by creating personalized outbound"
result= vectorstoredb.similarity_search(qurey)
result[0].page_content

'Shorten sales cycles by creating personalized outbound emails in your prospect’s language\nAccelerate time to market by quickly translating product descriptions into multiple languages\nImprove customer satisfaction by enabling support teams with digital assistants that answer questions in local languages'

In [13]:
from langchain_openai import ChatOpenAI
llm= ChatOpenAI(model="gpt-4o")

In [14]:
### Retrival chain, document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt= ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>

"""
)

document_chain= create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000153607D3B50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000153607D2950>, root_client=<openai.OpenAI object at 0x00000153607D0430>, root_async_client=<openai.AsyncOpenAI object at 0x00000153607D3280>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docu

In [17]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Shorten sales cycles by creating personalized outbound",
    "context":[Document(page_content="Shorten sales cycles by creating personalized outbound emails in your prospect’s language")]
})

'To shorten sales cycles, you can create personalized outbound emails that are written in the language of your prospect. This approach can enhance communication, make your outreach more relatable, and potentially speed up the decision-making process.'

However we want documents to first come from the retriver we just set up. That way, we can use the retriever tho dynamically select the most relavent document and pass those in for a give question.

In [18]:
## input ---> retriever----> vectorstoreDB

vectorstoredb

In [22]:
retriever_var = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever_var, document_chain)

In [23]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000152AF23CAF0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
            | Ch

In [25]:
## get the response from the LLM

response = retriever_chain.invoke({"input":"Shorten sales cycles by creating personalized outbound"})
response['answer']

'What are the benefits of using multilingual capabilities in business processes according to the provided context?\n\nThe benefits include shortening sales cycles by creating personalized outbound emails in the prospect’s language, accelerating time to market by quickly translating product descriptions into multiple languages, and improving customer satisfaction by enabling support teams with digital assistants that answer questions in local languages.'

In [26]:
response

{'input': 'Shorten sales cycles by creating personalized outbound',
 'context': [Document(metadata={'source': 'https://writer.com/blog/palmyra-llms-multilingual/?utm_source=google&utm_medium=paid-search&utm_campaign=multilingual&utm_content=multilingual-blog-post&utm_source=google&utm_medium=paid_search&utm_campaign=&utm_term=multilingual%20llm&utm_content=&hsa_acc=8779610535&hsa_cam=20957109982&hsa_grp=158242513895&hsa_ad=688126356904&hsa_src=g&hsa_tgt=kwd-2228024943017&hsa_kw=multilingual%20llm&hsa_mt=b&hsa_net=adwords&hsa_ver=3&gad_source=1&gclid=Cj0KCQjw99e4BhDiARIsAISE7P-8lapEODR3L9fXGY9r-xpddqrBDTD7xDQKVjKVolYNEU72v5i0WRcaAlA8EALw_wcB', 'title': 'Introducing multilingual capabilities with Palmyra LLMs - Writer', 'description': 'Palmyra, the Writer-built family of LLMs, now supports highly-accurate text generation and translation in over 30 languages. Find out more.', 'language': 'en-US'}, page_content='Shorten sales cycles by creating personalized outbound emails in your prospect